In [ ]:
import cv2
import sys
import time
import numpy as np
from scipy.spatial import distance as dist

# --- 1. Fungsi Bantuan ---
def get_centroid(bbox):
    """Menghitung titik tengah (centroid) dari sebuah bounding box."""
    (x, y, w, h) = bbox
    return (int(x + w / 2), int(y + h / 2))

def estimate_distance(pixel_width, known_width_cm, focal_length):
    """Fungsi untuk mengestimasi jarak."""
    if pixel_width == 0:
        return 0
    return (known_width_cm * focal_length) / pixel_width

# --- 2. Inisialisasi & Pilihan Pengguna ---
print("--- Sistem Deteksi dan Pelacakan Objek Dinamis ---")
print("Pilih objek yang ingin dideteksi:")
print("1: Manusia (Seluruh Tubuh)")
print("2: Hewan (Wajah Kucing)")
object_choice = input("Masukkan pilihan (1 atau 2): ")

# Variabel konfigurasi yang akan diatur berdasarkan pilihan
cascade_path = ""
object_label = ""
MIN_SIZE = (0, 0)
KNOWN_WIDTH_CM = 0.0

if object_choice == '1':
    cascade_path = r'E:\INF\Semester 5\Viskom\materi\kode\cascades\haarcascade_fullbody.xml'
    object_label = "Orang"
    MIN_SIZE = (40, 80)
    KNOWN_WIDTH_CM = 50.0 # Perkiraan lebar bahu orang dewasa
elif object_choice == '2':
    cascade_path = r'E:\INF\Semester 5\Viskom\materi\kode\cascades\haarcascade_frontalcatface.xml'
    object_label = "Kucing"
    MIN_SIZE = (30, 30)
    KNOWN_WIDTH_CM = 12.0 # Perkiraan lebar wajah kucing
else:
    print("Pilihan tidak valid. Keluar.")
    sys.exit()

# Muat classifier Haar Cascade yang dipilih
object_cascade = cv2.CascadeClassifier(cascade_path)

if object_cascade.empty():
    print(f"Error: Tidak dapat memuat file cascade dari: {cascade_path}")
    print("Pastikan file yang sesuai (haarcascade_fullbody.xml atau haarcascade_frontalcatface.xml) ada di folder ini.")
    sys.exit()

# Pilihan sumber video
video_path = input(f"Masukkan nama file video untuk deteksi '{object_label}': ")
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Tidak dapat membuka file video di path: {video_path}")
    sys.exit()

# --- Variabel untuk Multi-Tracking ---
trackers = {}
object_centroids = {}
object_timers = {}
next_object_id = 0
MAX_DISTANCE = 75 # Naikkan sedikit untuk deteksi tubuh yang kurang presisi
MAX_DISAPPEARED = 20
disappeared_frames = {}

FOCAL_LENGTH = 800 # Sesuaikan focal length jika perlu
frame_count = 0
DETECTION_INTERVAL = 10

print(f"\nSistem dimulai untuk mendeteksi '{object_label}'...")
print("Tekan 'q' untuk keluar.")

# --- 3. Loop Utama ---
while True:
    ret, frame = cap.read()
    if not ret:
        print("Video selesai.")
        break

    frame = cv2.resize(frame, (800, 600))
    (H, W) = frame.shape[:2]

    # --- 4. Tahap Deteksi Periodik ---
    if frame_count % DETECTION_INTERVAL == 0:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Deteksi objek menggunakan cascade yang dipilih
        detected_objects = object_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=3, minSize=MIN_SIZE)

        # --- Logika Pendaftaran dan Pencocokan Objek ---
        detected_centroids = [get_centroid(bbox) for bbox in detected_objects]
        
        if len(trackers) == 0:
            for bbox in detected_objects:
                centroid = get_centroid(bbox)
                tracker = cv2.TrackerCSRT_create()
                tracker.init(frame, tuple(bbox))
                
                trackers[next_object_id] = tracker
                object_centroids[next_object_id] = centroid
                object_timers[next_object_id] = time.time()
                disappeared_frames[next_object_id] = 0
                print(f"{object_label} baru terdeteksi. ID: {next_object_id}")
                next_object_id += 1
        else:
            object_ids = list(object_centroids.keys())
            previous_centroids = np.array(list(object_centroids.values()))
            
            if len(detected_centroids) > 0:
                D = dist.cdist(previous_centroids, detected_centroids)
                
                rows = D.min(axis=1).argsort()
                cols = D.argmin(axis=1)[rows]
                
                used_rows, used_cols = set(), set()
                for (row, col) in zip(rows, cols):
                    if row in used_rows or col in used_cols:
                        continue
                    if D[row, col] > MAX_DISTANCE:
                        continue
                    
                    object_id = object_ids[row]
                    disappeared_frames[object_id] = 0
                    used_rows.add(row)
                    used_cols.add(col)

                unused_cols = set(range(len(detected_objects))).difference(used_cols)
                for col in unused_cols:
                    bbox = detected_objects[col]
                    centroid = get_centroid(bbox)
                    tracker = cv2.TrackerCSRT_create()
                    tracker.init(frame, tuple(bbox))
                    
                    trackers[next_object_id] = tracker
                    object_centroids[next_object_id] = centroid
                    object_timers[next_object_id] = time.time()
                    disappeared_frames[next_object_id] = 0
                    print(f"{object_label} baru terdeteksi. ID: {next_object_id}")
                    next_object_id += 1

    # --- 5. Tahap Pelacakan ---
    object_ids_to_update = list(trackers.keys())
    for object_id in object_ids_to_update:
        tracker = trackers[object_id]
        success, bbox = tracker.update(frame)
        
        if success:
            object_centroids[object_id] = get_centroid(bbox)
            disappeared_frames[object_id] = 0
            
            (x, y, w, h) = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            distance_cm = estimate_distance(w, KNOWN_WIDTH_CM, FOCAL_LENGTH)
            duration = time.time() - object_timers[object_id]
            
            id_text = f"{object_label} ID: {object_id}"
            duration_text = f"Durasi: {int(duration)}s"
            dist_text = f"Est. Jarak: {distance_cm:.1f} cm"

            cv2.putText(frame, id_text, (x, y - 35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(frame, duration_text, (x, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(frame, dist_text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        else:
            disappeared_frames[object_id] += 1
            if disappeared_frames[object_id] > MAX_DISAPPEARED:
                print(f"Menghapus {object_label} ID {object_id} karena hilang.")
                del trackers[object_id]
                del object_centroids[object_id]
                del disappeared_frames[object_id]
                del object_timers[object_id]

    # --- 6. Tampilkan Info Tambahan ---
    count_text = f"Jumlah {object_label}: {len(trackers)}"
    cv2.putText(frame, count_text, (10, H - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    cv2.imshow(f"Sistem Pelacakan - Mode {object_label}", frame)
    
    frame_count += 1

    key = cv2.waitKey(25) & 0xFF
    if key == ord('q'):
        break

# --- 7. Cleanup ---
cap.release()
cv2.destroyAllWindows()

--- Sistem Deteksi dan Pelacakan Objek Dinamis ---
Pilih objek yang ingin dideteksi:
1: Manusia (Seluruh Tubuh)
2: Hewan (Wajah Kucing)

Sistem dimulai untuk mendeteksi 'Kucing'...
Tekan 'q' untuk keluar.
Kucing baru terdeteksi. ID: 0
